In [1]:
from py2neo import Graph
import pandas as pd

In [2]:
# connect to the graph -> it should be running already
graph = Graph(scheme="bolt", host="localhost", port=7687, auth=('neo4j', '12345'))

# connect to test database, only accessible inside ipf network
# pw = ''
# graph = Graph("bolt://neo4j.loc.ipfdd.de/newsocket",auth=('neo4j',pw))


In [3]:
import neo2BFMPolymer as neo2BFM

In [4]:
# simulation project function
def insertCodmucSimRunInGraph(myNeo4Polymer, simProject, simRunName, filepath, generation, spacerLength, graftedChainLength):
    '''insert all information of a codendrimer simulation Run (Codmuc) to the database'''
    myNeo4Polymer.addSimulationRunToSimulationProject(currentSimRun,currentProject)
    myNeo4Polymer.addBFMFileToDatabase(currentSimRun,filepath)
    #myNeo4Polymer.addDendrimerGenerationToSimulationRun(currentSimRun,generation)
    #myNeo4Polymer.addDendrimerSpacerLengthToSimulationRun(currentSimRun,spacer)
    myNeo4Polymer.addGraftedChainLengthToSimulationRun(currentSimRun,graftedChainLength)

# this ALWAYS deletes the full graph! Use this function with care
graph.delete_all()

# setup instacne of neo4j - BFM file interface
myNeoPolymers = neo2BFM.neo2BFMPolymer(graph)

In [ ]:
# -------------------------------------------------------------------------------------------------- #
# -------------------------------------------------------------------------------------------------- #
# Linear dendritic copolymers
### -- preparation of project -- ###
# add simulation project:
currentProject = "Linear-Dendritic Copolymer Solutions"
myNeoPolymers.addNewSimulationProject(currentProject)
# add grafted chain as polymer:
myNeoPolymers.addNewPolymer("Grafted Chain")
# add dendrimer as polymer:
myNeoPolymers.addNewPolymer("Dendrimer")
# connect Dendrimers:
myNeoPolymers.connectSimulationToPolymer("Dendrimer", currentProject)
# connect Grafted Chain:
myNeoPolymers.connectSimulationToPolymer("Grafted Chain", currentProject)


### -- read in all information of simRun 1 -- ###
generation = 3
spacer = 4
numMolecules = 27
graftedChainLength = 4
currentSimRun = "codmuc_micelle_g{}_s{}_m{}_l{}_e0.8_b128".format(generation,spacer,numMolecules,graftedChainLength)
filepath = "bfm_files/{}_1425000000_lastconfig.bfm".format(currentSimRun)
# use simulation project function:
insertCodmucSimRunInGraph(myNeoPolymers, currentProject, currentSimRun, filepath, generation, spacer, graftedChainLength)
# add radius of gyration results
filepath = "rg_files/{}_rg.dat".format(currentSimRun)
myNeoPolymers.addCODMUCRgTensorFileToDatabase(currentSimRun,filepath)

### -- read in all information of simRun 2 -- ###
generation = 4
spacer = 4
numMolecules = 12
graftedChainLength = 4
currentSimRun = "codmuc_micelle_g{}_s{}_m{}_l{}_e0.8_b128".format(generation,spacer,numMolecules,graftedChainLength)
filepath = "bfm_files/{}_2225000000_lastconfig.bfm".format(currentSimRun)
# use simulation project function:
insertCodmucSimRunInGraph(myNeoPolymers, currentProject, currentSimRun, filepath, generation, spacer, graftedChainLength)
# add radius of gyration results
filepath = "rg_files/{}_rg.dat".format(currentSimRun)
myNeoPolymers.addCODMUCRgTensorFileToDatabase(currentSimRun,filepath)


In [ ]:
# -------------------------------------------------------------------------------------------------- #
# -------------------------------------------------------------------------------------------------- #
# Ring opening Polymerisation
### -- preparation of project -- ###
# add simulation project:
currentProject = "Ring-Opening Polymerisation"
myNeoPolymers.addNewSimulationProject(currentProject)
# add ring polymer as polymer:
myNeoPolymers.addNewPolymer("Ring Polymer")
# connect Ring Polymer:
myNeoPolymers.connectSimulationToPolymer("Ring Polymer", currentProject)

### -- read in all information of simRun 1 -- ###
#RingMelt_N128_Phi05_Box128_EA5_EB17_MCS0_GPU
ring_size = 128
volume_fraction = "05"
activation_energy = 5
breaking_energy = 17

currentSimRun = "RingMelt_N{}_Phi{}_Box128_EA{}_EB{}_MCS0_GPU".format(ring_size,volume_fraction,activation_energy,breaking_energy)
filepath = "bfm_files/{}.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)

### -- read in all information of simRun 2 -- ###
#RingMelt_N128_Phi05_Box128_EA6_EB19_MCS0_GPU.bfm
ring_size = 128
volume_fraction = "05"
activation_energy = 6
breaking_energy = 19

currentSimRun = "RingMelt_N{}_Phi{}_Box128_EA{}_EB{}_MCS0_GPU".format(ring_size,volume_fraction,activation_energy,breaking_energy)
filepath = "bfm_files/{}.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)

In [6]:
# -------------------------------------------------------------------------------------------------- #
# -------------------------------------------------------------------------------------------------- #
# Single Dendrimer Variable solvent Quality
### -- preparation of project -- ###
# add simulation project:
currentProject = "Single Dendrimers Solvent Quality"
myNeoPolymers.addNewSimulationProject(currentProject)
# connect Dendrimer:
myNeoPolymers.connectSimulationToPolymer("Dendrimer", currentProject)

### -- read in all information of simRun 1 -- ###
#dendr_g4_s4_b64_solv_e05.bfm
generation = 4
spacer = 4
nn_ineraction = "05" # = 0.5 k_B T

currentSimRun = "dendr_g{}_s{}_b64_solv_e{}".format(generation,spacer,nn_ineraction)
filepath = "bfm_files/{}.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)
# add radius of gyration file
filepath = "rg_files/{}_rg_mol.dat".format(currentSimRun)
myNeoPolymers.addSingleDendrimerRgTFileToDatabase(currentSimRun, filepath)

### -- read in all information of simRun 1 -- ###
#dendr_g4_s4_b64_solv_e08.bfm
generation = 4
spacer = 4
nn_ineraction = "80" # = 0.8 k_B T

currentSimRun = "dendr_g{}_s{}_b64_solv_e{}".format(generation,spacer,nn_ineraction)
filepath = "bfm_files/{}.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)
# add radius of gyration file
filepath = "rg_files/{}_rg_mol.dat".format(currentSimRun)
myNeoPolymers.addSingleDendrimerRgTFileToDatabase(currentSimRun, filepath)

reached mcs command, stop reading. last mcs=  80000000
LeMonADEFeature with name FeatureBox already connected to Parameter with name BoxSize and value [64, 64, 64]
LeMonADEFeature with name FeatureBox already connected to Parameter with name Periodicity and value [1, 1, 1]
LeMonADEFeature with name FeatureSystemInformationDendrimer already connected to Parameter with name DendrimerGeneration and value 4
LeMonADEFeature with name FeatureSystemInformationDendrimer already connected to Parameter with name DendrimerSpacerLength and value 4
LeMonADEFeature with name FeatureSystemInformationDendrimer already connected to Parameter with name DendrimerCoreFunctionality and value 3
LeMonADEFeature with name FeatureSystemInformationDendrimer already connected to Parameter with name DendrimerBranchingPointFunctionality and value 3
path DESKTOP-7NTPDAA: C:/Users/wenge/Documents/Doktorarbeit/neo4j/neo4BFMPolymers/rg_files/dendr_g4_s4_b64_solv_e05_rg_mol.dat is added to dendr_g4_s4_b64_solv_e05 in p

True

In [ ]:
# -------------------------------------------------------------------------------------------------- #
# -------------------------------------------------------------------------------------------------- #
# Single Dendrimer in Polymer Solution Umbrella Sampling
### -- preparation of project -- ###
# add simulation project:
currentProject = "Single Dendrimer-Linear Chain Solution Umbrella"
myNeoPolymers.addNewSimulationProject(currentProject)
# add Linear Chain as polymer:
myNeoPolymers.addNewPolymer("Linear Chain")
# connect Polymers:
myNeoPolymers.connectSimulationToPolymer("Dendrimer", currentProject)
myNeoPolymers.connectSimulationToPolymer("Linear Chain", currentProject)

### -- read in all information of simRun 1 -- ###
# dend_chain_spring_g3_s4_l32_b64_d05_r08_k1.0.bfm
generation = 3
spacer = 4
linear_chain_length = 32
spring_lenght = "08"
spring_const = "1.0"
vol_frac = "05" # = 0.5 k_B T

currentSimRun = "dend_chain_spring_g{}_s{}_l{}_b64_d{}_r{}_k{}".format(generation,spacer,linear_chain_length,vol_frac,spring_lenght,spring_const)
filepath = "bfm_files/{}.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)

### -- read in all information of simRun 2 -- ###
# dend_chain_spring_g3_s4_l32_b64_d05_r012_k1.0.bfm
generation = 3
spacer = 4
linear_chain_length = 32
spring_lenght = "012"
spring_const = "1.0"
vol_frac = "05" # = 0.5 k_B T

currentSimRun = "dend_chain_spring_g{}_s{}_l{}_b64_d{}_r{}_k{}".format(generation,spacer,linear_chain_length,vol_frac,spring_lenght,spring_const)
filepath = "bfm_files/{}.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)

### -- read in all information of simRun 3 -- ###
# dend_chain_spring_g3_s4_l32_b64_d05_r020_k1.0.bfm
generation = 3
spacer = 4
linear_chain_length = 32
spring_lenght = "020"
spring_const = "1.0"
vol_frac = "05" # = 0.5 k_B T

currentSimRun = "dend_chain_spring_g{}_s{}_l{}_b64_d{}_r{}_k{}".format(generation,spacer,linear_chain_length,vol_frac,spring_lenght,spring_const)
filepath = "bfm_files/{}.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)

In [ ]:
# -------------------------------------------------------------------------------------------------- #
# -------------------------------------------------------------------------------------------------- #
# Linear chains
### -- preparation of project -- ###
# add simulation project:
currentProject = "Linear Chain Solution"
myNeoPolymers.addNewSimulationProject(currentProject)
# connect Linear chain:
myNeoPolymers.connectSimulationToPolymer("Linear Chain", currentProject)

### -- read in all information of simRun 1 -- ###
#linchains_n32_b64_d0.24_lastconfig.bfm
chain_length = 32
box_size = 64
volume_fraction = "0.24"

currentSimRun = "linchains_n{}_b{}_d{}".format(chain_length,box_size,volume_fraction)
filepath = "bfm_files/{}_lastconfig.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)
# add additional info
myNeoPolymers.addLinearChainLengthToSimulationRun(currentSimRun,chain_length)
myNeoPolymers.addVolumeFractionToSimulationRun(currentSimRun,volume_fraction)
# add radius of gyration
filepath = "rg_files/{}_rg.dat".format(currentSimRun)
myNeoPolymers.addLinearPolymerSolutionRgFileToDatabase(currentSimRun,filepath)

### -- read in all information of simRun 2 -- ###
#linchains_n32_b64_d0.36_lastconfig.bfm
chain_length = 32
box_size = 64
volume_fraction = "0.36"

currentSimRun = "linchains_n{}_b{}_d{}".format(chain_length,box_size,volume_fraction)
filepath = "bfm_files/{}_lastconfig.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)
# add additional info
myNeoPolymers.addLinearChainLengthToSimulationRun(currentSimRun,chain_length)
myNeoPolymers.addVolumeFractionToSimulationRun(currentSimRun,volume_fraction)
# add radius of gyration
filepath = "rg_files/{}_rg.dat".format(currentSimRun)
myNeoPolymers.addLinearPolymerSolutionRgFileToDatabase(currentSimRun,filepath)

In [ ]:
# test some functions

# this ALWAYS deletes the full graph! Use this function with care
#graph.delete_all()

myNeoPolymers = neo2BFMPolymer(graph)
print("add linear chain as polymer: ", myNeoPolymers.addNewPolymer("Linear Chain"))
print("add dendrimers as polymer: ", myNeoPolymers.addNewPolymer("Dendrimer"))
print("add star polymer as polymer: ", myNeoPolymers.addNewPolymer("Star Polymer"))
print("add comb polymer as polymer: ", myNeoPolymers.addNewPolymer("Comb Polymer"))

print("add Linear Polymer Melt as SimulationProject: ", myNeoPolymers.addNewSimulationProject("Linear Polymer Melt"))
print("add Single Linear Chain as SimulationProject: ", myNeoPolymers.addNewSimulationProject("Single Linear Chain"))
print("add Single Dendrimer as SimulationProject: ", myNeoPolymers.addNewSimulationProject("Single Dendrimer"))
print("add Mixture Dendrimer and Linear Chains as Simulation: ", myNeoPolymers.addNewSimulationProject("Mixture Dendrimer and Linear Chains"))

print("connect polymer and Melt", myNeoPolymers.connectSimulationToPolymer("Linear Chain", "Linear Polymer Melt"))
print("connect polymer and single chain", myNeoPolymers.connectSimulationToPolymer("Linear Chain", "Single Linear Chain"))
print("connect dendrimers", myNeoPolymers.connectSimulationToPolymer("Dendrimer", "Single Dendrimer"))
print("connect dendrimers", myNeoPolymers.connectSimulationToPolymer("Dendrimer", "Mixture Dendrimer and Linear Chains"))
print("connect dendrimers", myNeoPolymers.connectSimulationToPolymer("Linear Chain", "Mixture Dendrimer and Linear Chains"),"\n")

print("add SimulationRun single dendrimers run 1", myNeoPolymers.addSimulationRunToSimulationProject("Single Dendrimers Solvent Quality","Single Dendrimer"))
print("connect BoxSize", myNeoPolymers.addBoxSizeToSimulationRun("Single Dendrimers Solvent Quality", 64, 64, 64))
print("connect Periodicity", myNeoPolymers.addPeriodicityToSimulationRun("Single Dendrimers Solvent Quality", True, True, True))
print("connect DendrimerGeneration", myNeoPolymers.addDendrimerGenerationToSimulationRun("Single Dendrimers Solvent Quality", 5))
print("connect DendrimerSpacerLength", myNeoPolymers.addDendrimerSpacerLengthToSimulationRun("Single Dendrimers Solvent Quality", 2))
print("connect NNInteraction", myNeoPolymers.addNNInteractionToSimulationRun("Single Dendrimers Solvent Quality", 0.8),"\n")

print("add SimulationRun single chains run 1", myNeoPolymers.addSimulationRunToSimulationProject("Single Linear Chain At Wall","Single Linear Chain"))
print("connect BoxSize", myNeoPolymers.addBoxSizeToSimulationRun("Single Linear Chain At Wall", 64, 64, 64))
print("connect Periodicity", myNeoPolymers.addPeriodicityToSimulationRun("Single Linear Chain At Wall", True, True, False))
print("connect ChainLength", myNeoPolymers.addLinearChainLengthToSimulationRun("Single Linear Chain At Wall", 128))
print("add Result node Rg",myNeoPolymers.addResultRadiusOfGyration("Single Linear Chain At Wall",46.8),"\n")